In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Train

## 一、Preprocessing

###建立正確label資料

In [3]:
df_train_label=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/TrainLabel.csv')
#check 887 sets
df_train_label[(df_train_label['Test']==887)|(df_train_label['Reference']==887)]

,Test,Reference
650,864,887
735,924,887


In [5]:
#remove 650&735 
indexnum=735 
df_train_label.drop(indexnum,inplace=True)
df_train_label[730:740]  

,Test,Reference
731,924,805
732,924,806
733,924,838
734,924,858
736,924,906
737,924,1049
738,924,1054
739,933,916
740,936,442
741,936,850


In [6]:
df_train_label.reset_index(drop=True,inplace=True)
df_train_label['comb']=df_train_label['Test'].astype(str)+'_'+df_train_label['Reference'].astype(str)
#save
df_train_label.to_pickle('Label_Yes_1381')
df_train_label.head()

,Test,Reference,comb
0,3,415,3_415
1,3,649,3_649
2,9,5,9_5
3,25,32,25_32
4,25,41,25_41


###整理關鍵字字庫，並產生corpuslist

In [7]:
df1 = pd.read_excel(io='/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/keywords/02pest.list.xlsx', header = None)
df2 = pd.read_excel(io='/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/keywords/02crop.list.xlsx', header = None)
df3 = pd.read_excel(io='/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/keywords/02chem.list.xlsx', header = None)

list1=[ele for ele in df1[0][:]]
list1_1=[ele for ele in df1[1][:]if pd.notnull(ele)]
list1_2=[ele for ele in df1[2][:] if pd.notnull(ele)]
list1_3=[ele for ele in df1[3][:]if pd.notnull(ele)]
list1_4=[ele for ele in df1[4][:]if pd.notnull(ele)]
list1_5=[ele for ele in df1[5][:]if pd.notnull(ele)]
listpest=list1+list1_1+list1_2+list1_3+list1_4+list1_5
listpest[:10]

list2=[ele for ele in df2[0][:]]
list2_1=[ele for ele in df2[1][:]if pd.notnull(ele)]
list2_2=[ele for ele in df2[2][:] if pd.notnull(ele)]
list2_3=[ele for ele in df2[3][:]if pd.notnull(ele)]
listcrop=list2+list2_1+list2_2+list2_3
listcrop[:10]

list3=[ele for ele in df3[0][:]]
list3_1=[ele for ele in df3[1][:]if pd.notnull(ele)]
list3_2=[ele for ele in df3[2][:] if pd.notnull(ele)]
list3_3=[ele for ele in df3[3][:]if pd.notnull(ele)]
list3_4=[ele for ele in df3[4][:]if pd.notnull(ele)]
list3_5=[ele for ele in df3[5][:]if pd.notnull(ele)]
list3_6=[ele for ele in df3[6][:]if pd.notnull(ele)]
listchem=list3+list3_1+list3_2+list3_3+list3_4+list3_5+list3_6
listchem[:10]

with open('chemcorpus.txt', 'w') as f:
    for item in listchem:
        f.write("%s\n" % item)
with open('cropcorpus.txt', 'w') as f:
    for item in listcrop:
        f.write("%s\n" % item)  
with open('pestcorpus.txt', 'w') as f:
    for item in listpest:
        f.write("%s\n" % item)

###彙整所有文章庫，轉成dataframe

In [8]:
from collections import defaultdict
from pathlib import Path

my_dir_path = "/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/dataTrainComplete"

results = defaultdict(list)
for file in Path(my_dir_path).iterdir():
    with open(file, "r") as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())
df = pd.DataFrame(results)
print(f'檔案大小{df.shape}')
df.head()

檔案大小(560, 2)


,file_name,text
0,1054.txt,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...
1,1015.txt,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...
2,1082.txt,"本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭縣政府、行政院農業委員會動植..."
3,1074.txt,氣候溫暖且不穩定，臺南區農業改良場籲加強注意稻熱病的發生。臺南區農改場、防檢局及田邊好幫手關...
4,1111.txt,葡萄露菌病已開始發生，台中場籲請農民加強防治\n台灣主要葡萄產區在彰化縣、台中縣及南投縣，由...


刪除文章887, 969

In [9]:
df[(df['file_name']=='887.txt')|(df['file_name']=='969.txt')]

,file_name,text
529,887.txt,花蓮、宜蘭地區96年4月份作物病蟲疫情發生預測\n請參閱附加檔案或網站http://www....
556,969.txt,本局發佈「全國滅鼠週」農地野鼠防除成果新聞稿\n動植物防疫檢疫局於本（九十）年三月三十日至四...


In [10]:
df.drop(529,inplace=True)
df.drop(556,inplace=True)
df.reset_index(drop=True, inplace=True)

標註文章編號為int

In [11]:
#preprocessing of header of df
txtnumlist=[i for i in df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

In [12]:
df['name_num']=np.nan
for i in range(df.shape[0]):
  df['name_num'][i]=txtnumlist[i]
#改變欄位
df = df[['file_name','name_num','text']]
df.to_pickle('article558_text')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,file_name,name_num,text
0,1054.txt,1054.0,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...
1,1015.txt,1015.0,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...
2,1082.txt,1082.0,"本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭縣政府、行政院農業委員會動植..."
3,1074.txt,1074.0,氣候溫暖且不穩定，臺南區農業改良場籲加強注意稻熱病的發生。臺南區農改場、防檢局及田邊好幫手關...
4,1111.txt,1111.0,葡萄露菌病已開始發生，台中場籲請農民加強防治\n台灣主要葡萄產區在彰化縣、台中縣及南投縣，由...


###同義詞取代
多一個col:'txt_rep' 放取代後的文章

In [13]:
combine_dict = {}
# Replace_Words.txt是同義詞表，每行是一系列同義詞，用空格分割
replace_words = "/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/Replace_Words.txt"
for line in open(replace_words, "r", encoding='utf-8'):
  seperate_word = line.strip().split(" ")
  num = len(seperate_word)
  for i in range(1, num):
    combine_dict[seperate_word[i]] = seperate_word[0]
    #print(seperate_word)

combine_dict

{'10月': '十月',
 '11月': '十一月',
 '12月': '十二月',
 '1月': '一月',
 '2月': '二月',
 '3月': '三月',
 '4-4式波爾多液': '波爾多',
 '4月': '四月',
 '5月': '五月',
 '6月': '六月',
 '7月': '七月',
 '8月': '八月',
 '9月': '九月',
 '丁基加保乳劑': '丁基加保扶',
 '丁基加保扶': '丁基加保扶',
 '丁基加保扶乳劑': '丁基加保扶',
 '丁基加保扶可濕性粉劑': '丁基加保扶',
 '丁基加保扶水基乳劑': '丁基加保扶',
 '丁基加保扶粒劑': '丁基加保扶',
 '丁香': '晚崙西亞',
 '三得芬乳劑': '三得芬',
 '三星鄉': '宜蘭',
 '三氟派瑞水懸劑': '三氟派瑞',
 '三氯松可溶性粉劑': '三氯松',
 '三泰隆乳劑': '三泰隆',
 '三賽唑可溼性粉劑': '三賽唑',
 '三賽唑可濕性粉': '三賽唑',
 '三賽唑可濕性粉劑': '三賽唑',
 '三賽唑水懸劑': '三賽唑',
 '丙基喜樂松乳劑': '丙基喜樂松',
 '丙基喜樂松粒劑': '丙基喜樂松',
 '中國梨': '梨',
 '中埔': '嘉義',
 '中寮鄉': '南投',
 '乃力松乳劑': '乃力松',
 '二水': '彰化',
 '二水鄉': '彰化',
 '二硫松粒劑': '二硫松',
 '五結鄉': '宜蘭',
 '亞托待克利水懸劑': '亞托待克利',
 '亞托敏水分散性粒劑': '亞托敏',
 '亞托敏水懸劑': '亞托敏',
 '亞水懸劑': '亞醌螨',
 '亞滅培可溶性粉劑': '亞滅培',
 '亞滅培水溶性粉劑': '亞滅培',
 '亞滅寧乳劑': '亞滅寧',
 '亞滅寧水基乳劑': '亞滅寧',
 '亞磷酸': '亞磷酸+氫氧化鉀',
 '亞磷酸氫氧化鉀中和之溶液': '亞磷酸+氫氧化鉀',
 '亞磷酸混和氫氧化鉀': '亞磷酸+氫氧化鉀',
 '亞磷酸混氫氧化鉀': '亞磷酸+氫氧化鉀',
 '亞磷酸稀釋液': '亞磷酸',
 '亞賜圃乳劑': '亞賜圃',
 '亞賜圃可濕性粉劑': '亞賜圃',
 '佈芬淨': '布芬淨',
 '佈賜芬蟎水懸劑': '佈賜芬螨',
 '佈飛松乳劑': '

In [14]:
df['textrep']=df['text']
for i in range(df.shape[0]):
  for key in combine_dict:
    if key in df['textrep'][i]:
      df['textrep'][i]=df['textrep'][i].replace(key,combine_dict[key])

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,file_name,name_num,text,textrep
0,1054.txt,1054.0,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...
1,1015.txt,1015.0,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...
2,1082.txt,1082.0,"本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭縣政府、行政院農業委員會動植...","本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭政府、行政院農業委員會動植物..."
3,1074.txt,1074.0,氣候溫暖且不穩定，臺南區農業改良場籲加強注意稻熱病的發生。臺南區農改場、防檢局及田邊好幫手關...,氣候溫暖且不穩定，臺南農業改良場籲加強注意稻熱病的發生。臺南農改場、防檢局及田邊好幫手關心您...
4,1111.txt,1111.0,葡萄露菌病已開始發生，台中場籲請農民加強防治\n台灣主要葡萄產區在彰化縣、台中縣及南投縣，由...,葡萄露菌病已開始發生，臺中場籲請農民加強防治\n台灣主要葡萄產區在彰化、臺中及南投，由於五月...


###抓取關鍵字, 關鍵字詞數放回df

In [15]:
listmixed=listcrop+listpest+listchem

In [16]:
df['keywords']=''
for i in range(df.shape[0]):
  keywords=''
  for j in listmixed:
    if j in df['textrep'][i]:
      keywords=keywords+' '+j
  df['keywords'][i]=keywords

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


計算關鍵字詞數

In [17]:
def wordcounter(string):
  word_list = string.split(' ')
  number_of_words = len(word_list)
  return number_of_words

In [18]:
keywords_count=[wordcounter(string) for string in df['keywords']]
df['keycount']=np.nan
for i in range(df.shape[0]):
  df['keycount'][i]=keywords_count[i]
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,file_name,name_num,text,textrep,keywords,keycount
0,1054.txt,1054.0,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...,苗栗地區發布水稻水象鼻蟲及螟蟲(二化螟蟲、三化螟蟲)蟲害防治警報，苗栗區農改場、防檢局及田邊...,水稻 雜草 螟蟲 二化螟 三化螟 水稻水象鼻蟲 螟蛾 象鼻蟲 加保扶 培丹 芬普尼 撲滅松...,20.0
1,1015.txt,1015.0,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...,稻熱病蠢蠢欲動、防檢局籲請農友加強防範\n目前水稻生長陸續進入分蘗期，由於最近氣候乍暖還寒，...,水稻 稻熱病 葉稻熱病,4.0
2,1082.txt,1082.0,"本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭縣政府、行政院農業委員會動植...","本縣瓜實蠅、甜菜夜蛾田間密度持續升溫,請農民加強防治工作。宜蘭政府、行政院農業委員會動植物...",甜菜夜蛾 瓜實蠅,3.0
3,1074.txt,1074.0,氣候溫暖且不穩定，臺南區農業改良場籲加強注意稻熱病的發生。臺南區農改場、防檢局及田邊好幫手關...,氣候溫暖且不穩定，臺南農業改良場籲加強注意稻熱病的發生。臺南農改場、防檢局及田邊好幫手關心您...,水稻 稻熱病 葉稻熱病 三賽唑 亞賜圃 喜樂克拉 護粒松 丙基喜樂松,9.0
4,1111.txt,1111.0,葡萄露菌病已開始發生，台中場籲請農民加強防治\n台灣主要葡萄產區在彰化縣、台中縣及南投縣，由...,葡萄露菌病已開始發生，臺中場籲請農民加強防治\n台灣主要葡萄產區在彰化、臺中及南投，由於五月...,葡萄 梅 露菌病 達滅芬 鋅錳克絕 銅滅達樂 甲鋅毆殺斯 賽座滅 滅達樂,10.0


In [19]:
txtkeywordslists=[]
for i in range(558):
  keywords=''
  for j in listmixed: 
    if j in df['textrep'][i]:
      keywords=keywords+' '+j
  txtkeywordslists.append(keywords)

txtkeywordslists[:10]

[' 水稻 雜草 螟蟲 二化螟 三化螟 水稻水象鼻蟲 螟蛾 象鼻蟲 加保扶 培丹 芬普尼 撲滅松 賽達松 益滅松 免扶克 丁基加保扶 芬殺螨 芬殺 丁基加保扶',
 ' 水稻 稻熱病 葉稻熱病',
 ' 甜菜夜蛾 瓜實蠅',
 ' 水稻 稻熱病 葉稻熱病 三賽唑 亞賜圃 喜樂克拉 護粒松 丙基喜樂松',
 ' 葡萄 梅 露菌病 達滅芬 鋅錳克絕 銅滅達樂 甲鋅毆殺斯 賽座滅 滅達樂',
 ' 水稻 瘤野螟 白葉枯病 陶斯松 賽洛寧 鏈四環黴素 克枯爛 矽護芬 得芬諾 四環黴素',
 ' 水稻 梅 稻熱病 穗稻熱病 葉稻熱病 紋枯病 三賽唑 亞賜圃 嘉賜三賽唑 加普胺 賓克隆 福多寧 菲克利 克熱淨 護粒丹 維利熱必斯 熱必斯 加普',
 ' 水稻 桃子 梅 桃 稻熱病 葉稻熱病',
 ' 綠肥作物 斜紋夜蛾 性費洛蒙 費洛蒙',
 ' 水稻 稻熱病 穗稻熱病']

##二、Feature Engineering

###望重縮減文章分詞

In [20]:
# 計算詞頻
from sklearn.feature_extraction.text import CountVectorizer

# cvectorizer= CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
cvectorizer= CountVectorizer()
document_term_matrix = cvectorizer.fit_transform(txtkeywordslistss)  #tokenized_docs是已經抓出所有關鍵字的文章集合，以空白分隔關鍵字
tfs = document_term_matrix.toarray()
tokens = cvectorizer.get_feature_names_out()
# pd.DataFrame(tfs,columns=tokens)

NameError: ignored

In [ ]:
# 將詞頻小於5且不是keywords的分詞抓取出來，成為一個list
clean= []
for idx, x in enumerate(np.sum(tfs, axis=0)): #tfs來自上方
  if x< 5 and not tokens[idx] in dict_words:
    clean.append(idx)

NameError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算tf-idf
vectorizer = TfidfVectorizer()
document_term_matrix = vectorizer.fit_transform(tokenized_docs)
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names_out()
# pd.DataFrame(tfidfs,columns=tokens)

In [ ]:
# 將上方計算出來的tf-idf值，刪除clean裡面那些比較不具參考價值的分詞
tokens= np.delete(tokens, clean)
tfidfs= np.delete(tfidfs, clean, axis=1)
# pd.DataFrame(tfidfs,columns=tokens)

###關鍵字法

####比對抓取關鍵字存成list

In [20]:
def preparekeylist(df,corpuslist,namelist):
  for i in range(df.shape[0]):
    keywords=''
    for j in corpuslist:
      if j in df['textrep'][i]:
        keywords=keywords+' '+j
    namelist.append(keywords)
  return namelist

In [21]:
txtpestlist=[]
txtcroplist=[]
txtchemlist=[]

preparekeylist(df,listpest,txtpestlist)
preparekeylist(df,listcrop,txtcroplist)
preparekeylist(df,listchem,txtchemlist)
#preparekeylist(df,listloca,txtlocalist)
#preparekeylist(df,listweat,txtweatlist)

#txtcroplist[:10]

[' 加保扶 培丹 芬普尼 撲滅松 賽達松 益滅松 免扶克 丁基加保扶 芬殺螨 芬殺 丁基加保扶',
 '',
 '',
 ' 三賽唑 亞賜圃 喜樂克拉 護粒松 丙基喜樂松',
 ' 達滅芬 鋅錳克絕 銅滅達樂 甲鋅毆殺斯 賽座滅 滅達樂',
 ' 陶斯松 賽洛寧 鏈四環黴素 克枯爛 矽護芬 得芬諾 四環黴素',
 ' 三賽唑 亞賜圃 嘉賜三賽唑 加普胺 賓克隆 福多寧 菲克利 克熱淨 護粒丹 維利熱必斯 熱必斯 加普',
 '',
 ' 性費洛蒙 費洛蒙',
 '',
 '',
 '',
 ' 三賽唑 亞賜圃 護粒松 嘉賜三賽唑 撲殺熱 芬諾尼',
 '',
 ' 三賽唑 嘉賜三賽唑 富米熱斯 加普胺 加普',
 ' 性費洛蒙 蘇力菌 費洛蒙',
 ' 亞磷酸 微生物製劑 三賽唑 亞賜圃 嘉賜黴素 喜樂克拉 護粒松 亞磷酸+氫氧化鉀',
 '',
 '',
 ' 三賽唑 喜樂克拉 護粒松 撲殺熱 丙基喜樂松',
 ' 待克利 賓克隆 福多寧 菲克利 維利黴素 貝芬同 福拉比',
 ' 三賽唑 喜樂克拉 護粒松 撲殺熱',
 ' 亞磷酸 亞磷酸+氫氧化鉀 達滅芬 鋅錳克絕 氟比拔克 銅滅達樂 甲鋅毆殺斯 波爾多 滅達樂',
 ' 賜諾殺 克蠅 克蠅香 馬拉松 蛋白質水解物',
 ' 性費洛蒙 費洛蒙',
 ' 性費洛蒙 費洛蒙',
 '',
 '',
 ' 性費洛蒙 氰氮化鈣 費洛蒙',
 ' 加保扶 培丹 芬普尼 撲滅松 賽達松 益滅松 免扶克 丁基加保扶 芬殺螨 芬殺 丁基加保扶',
 ' 腐絕快得寧 免得爛 克收欣 福賽快得寧 快得寧 腐絕',
 '',
 ' 賽洛寧 芬化利',
 ' 鋅錳乃浦 免得爛 亞托敏 達滅芬 甲基鋅乃浦 鋅錳克絕 錳乃浦 鋅錳右滅達樂 賽座滅 滅達樂',
 ' 賽洛寧 待克利 菲克利 免得爛 扶吉胺 滅賜克 貝芬依滅列 賽普待克利 克熱淨 菲克利腐絕 克凡派 依滅列 克熱淨(烷苯磺酸鹽) 腐絕',
 ' 三賽唑 亞賜圃 喜樂克拉 護粒松 丙基喜樂松',
 '',
 '',
 ' 馬拉松 芬殺螨 蛋白質水解物 芬殺',
 ' 鋅錳乃浦 錳乃浦',
 ' 鋅錳乃浦 錳乃浦 普克利',
 ' 撲殺熱 鏈四環黴素 克枯爛 嘉賜克枯爛 四環黴素',
 ' 三賽唑 護粒松 嘉賜三賽唑 撲殺熱 加普胺 鏈四環黴素 克枯爛 賓克隆 福多寧 菲克利 

####embedding+cosine similarity (手動重複)

In [22]:
def Cosine_similarity(vec_a,vec_b):
  # Dot and norm
  dot = sum(a*b for a, b in zip(vec_a, vec_b))
  norm_a = sum(a*a for a in vec_a) ** 0.5
  norm_b = sum(b*b for b in vec_b) ** 0.5

  # Cosine similarity
  cos_sim = dot / (norm_a*norm_b)
  return cos_sim

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
#分次做：txtcroplist
document_term_matrix = vectorizer.fit_transform(txtcroplist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
df999=(pdkey==0).all(axis=1)
#df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in df999[df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of df
txtnumlist=[i for i in df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

df_sim_value1=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
#df_sim_value.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
#分次做：txtpestlist
document_term_matrix = vectorizer.fit_transform(txtpestlist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
df999=(pdkey==0).all(axis=1)
#df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in df999[df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of df
txtnumlist=[i for i in df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

df_sim_value2=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
#df_sim_value.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
#分次做：txtchemlist
document_term_matrix = vectorizer.fit_transform(txtchemlist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
df999=(pdkey==0).all(axis=1)
#df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in df999[df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of df
txtnumlist=[i for i in df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

df_sim_value3=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
#df_sim_value.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
#把df轉成table列表
#sim_value_crop, sim_value_pest, sim_value_chem
df1=df_sim_value1.where(np.triu(np.ones(df_sim_value1.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_crop')
df2=df_sim_value1.where(np.tril(np.ones(df_sim_value1.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_crop')
df3=df_sim_value2.where(np.triu(np.ones(df_sim_value2.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_pest')
df4=df_sim_value2.where(np.tril(np.ones(df_sim_value2.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_pest')
df5=df_sim_value3.where(np.triu(np.ones(df_sim_value3.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_chem')
df6=df_sim_value3.where(np.tril(np.ones(df_sim_value3.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_chem')
tablesim1 = pd.concat([df1,df2])
tablesim2 = pd.concat([df3,df4])
tablesim3 = pd.concat([df5,df6])
#tablesim
tablesim1.reset_index(drop=True, inplace=True)
tablesim2.reset_index(drop=True, inplace=True)
tablesim3.reset_index(drop=True, inplace=True)
#暫存各個分類檔案
tablesim1.to_pickle('tablesim_crop')
tablesim2.to_pickle('tablesim_pest')
tablesim3.to_pickle('tablesim_chem')

####整合以上數值，並加入所需特徵值

In [55]:
#合併
tabelcrop=pd.read_pickle('tablesim_crop')
tabelpest=pd.read_pickle('tablesim_pest')
tabelchem=pd.read_pickle('tablesim_chem')
tablesim=pd.concat([tabelcrop,tabelpest['sim_value_pest'],tabelchem['sim_value_chem'] ],axis=1)

#加入關鍵字詞數
tablesim['Test']=tablesim['Test'].astype(int)
tablesim['Reference']=tablesim['Reference'].astype(int)

tablesim['Test_key_cou']=np.nan
for i in range(tablesim.shape[0]):
  tablesim['Test_key_cou'][i]=df[df['name_num']==tablesim['Test'][i]]['keycount']

tablesim['Ref_key_cou']=np.nan
for i in range(tablesim.shape[0]):
  tablesim['Ref_key_cou'][i]=df[df['name_num']==tablesim['Reference'][i]]['keycount']

tablesim.head()

串流輸出內容已截斷至最後 5000 行。
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A

,Test,Reference,sim_value_crop,sim_value_pest,sim_value_chem,Test_key_cou,Ref_key_cou
0,1054,1015,0.539933,0.0,0.0,20.0,4.0
1,1054,1082,0.000000,0.0,0.0,20.0,3.0
2,1054,1074,0.539933,0.0,0.0,20.0,9.0
3,1054,1111,0.000000,0.0,0.0,20.0,10.0
4,1054,1114,0.539933,0.0,0.0,20.0,11.0


####Label Y(yes or no)

In [56]:
tablesim.reset_index(drop=True,inplace=True)
tablesim['comb']=tablesim['Test'].astype(str)+'_'+tablesim['Reference'].astype(str)
tablesim

,Test,Reference,sim_value_crop,sim_value_pest,sim_value_chem,Test_key_cou,Ref_key_cou,comb
0,1054,1015,0.539933,0.0,0.0,20.0,4.0,1054_1015
1,1054,1082,0.000000,0.0,0.0,20.0,3.0,1054_1082
2,1054,1074,0.539933,0.0,0.0,20.0,9.0,1054_1074
3,1054,1111,0.000000,0.0,0.0,20.0,10.0,1054_1111
4,1054,1114,0.539933,0.0,0.0,20.0,11.0,1054_1114
...,...,...,...,...,...,...,...,...
310801,898,940,0.000000,0.0,0.0,3.0,15.0,898_940
310802,898,933,0.000000,0.0,0.0,3.0,11.0,898_933
310803,898,95,0.000000,0.0,0.0,3.0,5.0,898_95
310804,898,951,0.000000,0.0,0.0,3.0,10.0,898_951


In [57]:
labelsumdf=pd.read_pickle('Label_Yes_1381')
labelsumdf

,Test,Reference,comb
0,3,415,3_415
1,3,649,3_649
2,9,5,9_5
3,25,32,25_32
4,25,41,25_41
...,...,...,...
1376,1382,921,1382_921
1377,1382,940,1382_940
1378,1382,949,1382_949
1379,1391,1392,1391_1392


In [58]:

labelsumdf=pd.read_pickle('Label_Yes_1381')
#開始比對，並標註上label。
tablesim['Label']=''
for i in labelsumdf['comb']:
  for j in tablesim['comb']:
    if i==j:
      Num=tablesim[tablesim['comb']==j].index.item()
      tablesim['Label'][Num]='Yes'
  
tablesim.replace('',"No",inplace=True)
tablesim.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Test,Reference,sim_value_crop,sim_value_pest,sim_value_chem,Test_key_cou,Ref_key_cou,comb,Label
0,1054,1015,0.539933,0.0,0.0,20.0,4.0,1054_1015,No
1,1054,1082,0.000000,0.0,0.0,20.0,3.0,1054_1082,No
2,1054,1074,0.539933,0.0,0.0,20.0,9.0,1054_1074,No
3,1054,1111,0.000000,0.0,0.0,20.0,10.0,1054_1111,No
4,1054,1114,0.539933,0.0,0.0,20.0,11.0,1054_1114,No


##三、Training datasets selection

###望重

In [ ]:
from itertools import permutations  # 計算排列的函式

rows= []
X= []
filename_list= list(df["file_name"])
length= len(filename_list)

for row in train_label.values:  # 整合上需要改train_label.values這個部份，這是抓取「label為yes」的組合
  # x即左方的檔案編號，y為右方
  x= row[0]
  y= row[1]
  # 由於相似的數量比不相似的數量少很多，所以以相似的sample數為基礎，只取相同數量的不相似的文章
  index_x= filename_list.index(x)
  index_y= filename_list.index(y)

  # rows.append({"Test":x, "Reference":y, "label":1})   # 這是產生組合和y_label用的csv所用的，由於上面已經先生成這內容，所以可以拿掉
  X.append(np.array([tfidfs[index_x], tfidfs[index_y]]))
    
  # 加入一個不相似的文章
  while True:
    temp_x= index_x+ np.random.randint(-45, 45)
    temp_y= index_y+ np.random.randint(-45, 45)

    # 判斷資料是否存在方法一
    if temp_x > 0 and temp_y > 0 and temp_x < length and temp_y < length:
      mask1= train_label["Test"] == filename_list[temp_x]
      mask2= train_label["Reference"] == filename_list[temp_y]
      if len(train_label[(mask1&mask2)]) == 0:
        break
    
  # rows.append({"Test":filename_list[temp_x], "Reference":filename_list[temp_y], "label":0})
  X.append(np.array([tfidfs[temp_x], tfidfs[temp_y]]))

###康琦

In [59]:
#隨機取1381筆No
No = tablesim[tablesim['Label']=='No']
order = No.sample(n=1381,\
                  )

#取出Yes
Yes = tablesim[tablesim['Label']=='Yes']
#合併
final = pd.concat([order,Yes])
final.reset_index(drop=True,inplace=True)

###Carol

In [35]:
label_df=pd.read_pickle('Label_Yes_1381')
label_df['comb2']=label_df['Reference'].astype(str)+'_'+label_df['Test'].astype(str)
df=pd.DataFrame(columns=['Test', 'Reference','comb','comb2'])
for i in label_df['comb2']:
  for j in label_df['comb']:
    if i==j:
      df_temp1=label_df[label_df['comb2']==i]
      df_temp2=label_df[label_df['comb']==j]
      df=pd.concat([df,df_temp1,df_temp2]) 
df_new = label_df.drop(df.index)
df_new.to_pickle('Label_diff')

In [36]:
#切割要train的data, 讓label:Yes, No平均
Labelyes=tablesim[tablesim['Label']=='Yes']
#從no當中挑選，yes組中反向為no的(Label_diff)皆列入
Label_diff=pd.read_pickle('Label_diff')
#挑出yes組中反向為no的 999筆
df_filter=pd.DataFrame(columns=['Test', 'Reference','sim_value_crop','sim_value_pest','sim_value_chem','Test_key_cou','Ref_key_cou','comb','Label']) 
for i in Label_diff['comb2']:
  for j in tablesim['comb']:
    if i==j:
      df_temp=tablesim[tablesim['comb']==j]
      df_filter=pd.concat([df_filter,df_temp]) 
#過濾999剩下的
Labelno=tablesim[tablesim['Label']=='No']
Labelno_drop = Labelno.drop(df_filter.index)
#取出相當量的no 取380篇
Labelno0_1=Labelno_drop[(Labelno_drop['sim_value_pest']==0)&(Labelno_drop['sim_value_crop']==0)].sample(115)
Labelno0_2=Labelno_drop[(Labelno_drop['sim_value_chem']==0)&(Labelno_drop['sim_value_pest']==0)].sample(115)
Labelno0_3=Labelno_drop[(Labelno_drop['sim_value_chem']==0)&(Labelno_drop['sim_value_pest']==0)&(Labelno_drop['sim_value_crop']==0)].sample(150)
frames = [Labelno0_1, Labelno0_2, Labelno0_3,df_filter] 
Label_no = pd.concat(frames)
#結合yes, no
traindata_bal=pd.concat([Labelyes, Label_no])
traindata_bal.reset_index(drop=True,inplace=True)
#save
traindata_bal.to_pickle('traindata_bal_1207')
traindata_bal

,Test,Reference,sim_value_crop,sim_value_pest,sim_value_chem,Test_key_cou,Ref_key_cou,comb,Label
0,1054,1049,1.000000,1.000000,1.000000,20.0,20.0,1054_1049,Yes
1,1054,806,1.000000,1.000000,1.000000,20.0,20.0,1054_806,Yes
2,1015,1000,1.000000,0.754908,0.000000,4.0,5.0,1015_1000,Yes
3,1015,1005,1.000000,0.754908,0.000000,4.0,5.0,1015_1005,Yes
4,1015,1007,1.000000,1.000000,0.000000,4.0,4.0,1015_1007,Yes
...,...,...,...,...,...,...,...,...,...
2753,170,1381,1.000000,0.421480,0.000000,9.0,4.0,170_1381,No
2754,362,1381,1.000000,0.745903,0.000000,5.0,4.0,362_1381,No
2755,921,1382,0.357740,1.000000,0.717435,14.0,9.0,921_1382,No
2756,940,1382,0.293893,1.000000,0.717435,15.0,9.0,940_1382,No


##四、ML: classification

###Split

In [60]:
#Data Preparation
from sklearn import preprocessing

#train_input 
X= np.array(final[['sim_value_crop','sim_value_pest','sim_value_chem','Test_key_cou','Ref_key_cou']])

# call LabelEncoder from the class: preprocessing
le = preprocessing.LabelEncoder()  

#train_target 
y = le.fit_transform(final['Label']) 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (2208, 5) (2208,)
Test set: (552, 5) (552,)


###採以下四種方式計算

In [61]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#####LogisticRegression

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)

yhat = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)

Ans4_J=jaccard_score(y_test, yhat,pos_label=0)
Ans4_F1=f1_score(y_test, yhat, average='weighted') 
Ans4_LL=log_loss(y_test, yhat_prob)

#####K Nearest Neighbor(KNN)

In [63]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#Training with function, set a K value loop
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

#show the best model
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

The best accuracy was with 0.9456521739130435 with k= 1


In [64]:
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = 1).fit(X_train,y_train)

yhat=neigh.predict(X_test)
Ans1_J=jaccard_score(y_test, yhat,pos_label=0)
Ans1_F1=f1_score(y_test, yhat, average='weighted') 

#####Decision Tree

In [65]:
from sklearn.tree import DecisionTreeClassifier
# modeling
Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
# Tree # it shows the default parameters
Tree.fit(X,y)

yhat=Tree.predict(X_test)
Ans2_J=jaccard_score(y_test, yhat,pos_label=0)
Ans2_F1=f1_score(y_test, yhat, average='weighted') 

####Support Vector Machine

In [66]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train) 

yhat=clf.predict(X_test)
Ans3_J=jaccard_score(y_test, yhat,pos_label=0)
Ans3_F1=f1_score(y_test, yhat, average='weighted') 

###用Jaccard & F1-score 評估四種方式

In [67]:
Ans_sheet= pd.DataFrame(np.array([[Ans1_J,Ans1_F1,None], [Ans2_J,Ans2_F1,None], [Ans3_J,Ans3_F1,None],[Ans4_J,Ans4_F1,Ans4_LL]]),
                   columns=['Jaccard', 'F1-score', 'LogLoss'],index=['KNN','Decision Tree','SVM','Logistic Regression'])
Ans_sheet

,Jaccard,F1-score,LogLoss
KNN,0.899666,0.945658,None
Decision Tree,0.937282,0.967397,None
SVM,0.894915,0.943855,None
Logistic Regression,0.90378,0.949281,0.28203


In [69]:
#儲存要使用的model
import joblib
#lr是一個LogisticRegression模型
joblib.dump(Tree, 'Tree_1214.model') #lr
#neigh = joblib.load('neigh.model')

['Tree_1214.model']

#Predict

##一、Preprocessing

In [9]:
from collections import defaultdict
from pathlib import Path

dir_path = "/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/dataPrivateComplete"

result = defaultdict(list)
for file in Path(dir_path).iterdir():
    with open(file, "r") as file_open:
        result["file_name"].append(file.name)
        result["text"].append(file_open.read())
Df = pd.DataFrame(result)
print(f'檔案大小{Df.shape}')
Df.head()

檔案大小(420, 2)


,file_name,text
0,101.txt,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...
1,1002.txt,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...
2,1013.txt,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區加強防治\n號稱行軍蟲的斜紋夜蛾目前已進入發生...
3,1059.txt,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...
4,1071.txt,溫室葡萄葉螨已開始發生，臺中區農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好...


In [10]:
#preprocessing of header of df
txtnumlist=[i for i in Df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

In [11]:
Df['name_num']=np.nan
for i in range(Df.shape[0]):
  Df['name_num'][i]=txtnumlist[i]
#改變欄位
Df = Df[['file_name','name_num','text']]
Df.to_pickle('article420_text')
Df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,file_name,name_num,text
0,101.txt,101.0,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...
1,1002.txt,1002.0,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...
2,1013.txt,1013.0,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區加強防治\n號稱行軍蟲的斜紋夜蛾目前已進入發生...
3,1059.txt,1059.0,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...
4,1071.txt,1071.0,溫室葡萄葉螨已開始發生，臺中區農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好...


In [12]:
combine_dict = {}
# Replace_Words.txt是同義詞表，每行是一系列同義詞，用空格分割
replace_words = "/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/Replace_Words.txt"
for line in open(replace_words, "r", encoding='utf-8'):
  seperate_word = line.strip().split(" ")
  num = len(seperate_word)
  for i in range(1, num):
    combine_dict[seperate_word[i]] = seperate_word[0]
    #print(seperate_word)

combine_dict

{'10月': '十月',
 '11月': '十一月',
 '12月': '十二月',
 '1月': '一月',
 '2月': '二月',
 '3月': '三月',
 '4-4式波爾多液': '波爾多',
 '4月': '四月',
 '5月': '五月',
 '6月': '六月',
 '7月': '七月',
 '8月': '八月',
 '9月': '九月',
 '丁基加保乳劑': '丁基加保扶',
 '丁基加保扶': '丁基加保扶',
 '丁基加保扶乳劑': '丁基加保扶',
 '丁基加保扶可濕性粉劑': '丁基加保扶',
 '丁基加保扶水基乳劑': '丁基加保扶',
 '丁基加保扶粒劑': '丁基加保扶',
 '丁香': '晚崙西亞',
 '三得芬乳劑': '三得芬',
 '三星鄉': '宜蘭',
 '三氟派瑞水懸劑': '三氟派瑞',
 '三氯松可溶性粉劑': '三氯松',
 '三泰隆乳劑': '三泰隆',
 '三賽唑可溼性粉劑': '三賽唑',
 '三賽唑可濕性粉': '三賽唑',
 '三賽唑可濕性粉劑': '三賽唑',
 '三賽唑水懸劑': '三賽唑',
 '丙基喜樂松乳劑': '丙基喜樂松',
 '丙基喜樂松粒劑': '丙基喜樂松',
 '中國梨': '梨',
 '中埔': '嘉義',
 '中寮鄉': '南投',
 '乃力松乳劑': '乃力松',
 '二水': '彰化',
 '二水鄉': '彰化',
 '二硫松粒劑': '二硫松',
 '五結鄉': '宜蘭',
 '亞托待克利水懸劑': '亞托待克利',
 '亞托敏水分散性粒劑': '亞托敏',
 '亞托敏水懸劑': '亞托敏',
 '亞水懸劑': '亞醌螨',
 '亞滅培可溶性粉劑': '亞滅培',
 '亞滅培水溶性粉劑': '亞滅培',
 '亞滅寧乳劑': '亞滅寧',
 '亞滅寧水基乳劑': '亞滅寧',
 '亞磷酸': '亞磷酸+氫氧化鉀',
 '亞磷酸氫氧化鉀中和之溶液': '亞磷酸+氫氧化鉀',
 '亞磷酸混和氫氧化鉀': '亞磷酸+氫氧化鉀',
 '亞磷酸混氫氧化鉀': '亞磷酸+氫氧化鉀',
 '亞磷酸稀釋液': '亞磷酸',
 '亞賜圃乳劑': '亞賜圃',
 '亞賜圃可濕性粉劑': '亞賜圃',
 '佈芬淨': '布芬淨',
 '佈賜芬蟎水懸劑': '佈賜芬螨',
 '佈飛松乳劑': '

In [13]:
Df['textrep']=Df['text']
for i in range(Df.shape[0]):
  for key in combine_dict:
    if key in Df['textrep'][i]:
      Df['textrep'][i]=Df['textrep'][i].replace(key,combine_dict[key])

Df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,file_name,name_num,text,textrep
0,101.txt,101.0,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...
1,1002.txt,1002.0,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...
2,1013.txt,1013.0,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區加強防治\n號稱行軍蟲的斜紋夜蛾目前已進入發生...,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區地區加強防治\n號稱斜紋夜蛾的斜紋夜蛾目前已進...
3,1059.txt,1059.0,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...
4,1071.txt,1071.0,溫室葡萄葉螨已開始發生，臺中區農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好...,溫室葡萄葉螨已開始發生，臺中農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好幫...


In [14]:
listmixed=listcrop+listpest+listchem

In [15]:
Df['keywords']=''
for i in range(Df.shape[0]):
  keywords=''
  for j in listmixed:
    if j in Df['textrep'][i]:
      keywords=keywords+' '+j
  Df['keywords'][i]=keywords

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
def wordcounter(string):
  word_list = string.split(' ')
  number_of_words = len(word_list)
  return number_of_words

In [17]:
keywords_count=[wordcounter(string) for string in Df['keywords']]
Df['keycount']=np.nan
for i in range(Df.shape[0]):
  Df['keycount'][i]=keywords_count[i]
Df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,file_name,name_num,text,textrep,keywords,keycount
0,101.txt,101.0,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...,發布本(107)年度第1期水稻二化螟蟲發生預報，請查照。\n本(107)年度三月下旬及四月上...,水稻 桃子 桃 螟蟲 二化螟 培丹 撲滅松,8.0
1,1002.txt,1002.0,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...,敏督利颱風過境，請嚴防農作物疫病蟲害蔓延\n九十三年七月九日動植物防疫檢疫局發布之新聞稿行政...,水稻 蔬菜 檬果 晚崙西亞 斜紋夜蛾 軟腐病 疫病 黑腐病 露菌病 稻熱病 果斑病 果實蠅,13.0
2,1013.txt,1013.0,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區加強防治\n號稱行軍蟲的斜紋夜蛾目前已進入發生...,斜紋夜蛾進入高峰警戒期 防檢局籲請雲嘉南地區地區加強防治\n號稱斜紋夜蛾的斜紋夜蛾目前已進...,蔬菜 豆類 雜糧 綠肥作物 斜紋夜蛾 性費洛蒙 費洛蒙,14.0
3,1059.txt,1059.0,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...,請宣導蕉農正確採行香蕉葉部病害防治工作，確保香蕉良好之品質及產量。\n一、近期時有冬雨，加上...,香蕉 鋅錳乃浦 錳乃浦 普克利,5.0
4,1071.txt,1071.0,溫室葡萄葉螨已開始發生，臺中區農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好...,溫室葡萄葉螨已開始發生，臺中農業改良場籲請農友及早預防，避免損失。臺中場、防檢局及田間好幫...,葡萄 葉螨 阿巴汀 畢芬寧 依殺螨 賽芬螨 賜派芬 芬殺螨 依殺 賽芬 芬殺,12.0


##二、Feature Engineering

In [18]:
def preparekeylist(Df,corpuslist,namelist):
  for i in range(Df.shape[0]):
    keywords=''
    for j in corpuslist:
      if j in Df['textrep'][i]:
        keywords=keywords+' '+j
    namelist.append(keywords)
  return namelist

In [19]:
Txtpestlist=[]
Txtcroplist=[]
Txtchemlist=[]

preparekeylist(Df,listpest,Txtpestlist)
preparekeylist(Df,listcrop,Txtcroplist)
preparekeylist(Df,listchem,Txtchemlist)
#preparekeylist(Df,listloca,txtlocalist)
#preparekeylist(Df,listweat,txtweatlist)

#txtcroplist[:10]

[' 培丹 撲滅松',
 '',
 ' 性費洛蒙 費洛蒙      ',
 ' 鋅錳乃浦 錳乃浦 普克利',
 ' 阿巴汀 畢芬寧 依殺螨 賽芬螨 賜派芬 芬殺螨 依殺 賽芬 芬殺',
 ' 性費洛蒙 費洛蒙',
 ' 加保扶 賽洛寧 賜諾特 丁基加保扶 滅賜克 丁基加保扶',
 ' 加保扶',
 '',
 '',
 ' 性費洛蒙 費洛蒙      ',
 ' 三賽唑 嘉賜三賽唑 富米熱斯 加普胺 加普',
 ' 亞托敏 錳乃浦      ',
 ' 耐克螺 聚乙醛',
 '',
 ' 賜諾殺 含毒蛋白質水解物 蛋白質水解物      ',
 ' 三賽唑',
 ' 三賽唑 亞賜圃 嘉賜黴素 甲基多保淨 嘉賜三賽唑 護粒三賽唑 保米熱斯 丙基喜樂松 富米熱斯 克熱賜圃 芬諾尼',
 '',
 ' 三賽唑 加保扶 甲基多保淨 嘉賜三賽唑 護粒三賽唑 保米熱斯 加普胺 培丹 免扶克 丁基加保扶 加普 丁基加保扶',
 ' 加保扶 芬普尼 丁基加保扶 丁基加保扶',
 ' 三賽唑 嘉賜黴素 護粒三賽唑 保米熱斯 加普胺 保米黴素 熱必斯 加普',
 ' 三賽唑 亞賜圃 喜樂克拉 護粒松 嘉賜三賽唑 撲殺熱 陶斯松 加普胺 培丹 芬普尼 撲滅松 加普',
 '',
 ' 護粒松 撲殺熱 加普胺 保米賜圃 矽酸爐渣 加普',
 ' 芬普尼 克凡派',
 ' 亞托敏 達滅芬 達滅克敏 鋅錳座賽胺',
 ' 三賽唑 喜樂克拉 護粒松 丙基喜樂松 芬諾尼',
 ' 三賽唑 亞賜圃 嘉賜黴素 護粒三賽唑',
 ' 加保扶 丁基加保扶 納乃得 滅大松 芬殺螨 芬殺 丁基加保扶',
 ' 亞磷酸 微生物製劑 三賽唑 亞賜圃 護粒松 嘉賜三賽唑 撲殺熱 加普胺 亞磷酸+氫氧化鉀 加普',
 ' 亞磷酸 三元硫酸銅 亞磷酸+氫氧化鉀 硫酸銅',
 ' 加保扶 陶斯松 賽洛寧 丁基加保扶 亞滅培 丁基加保扶',
 ' 扶吉胺 白列克敏 賽普護汰寧 護汰寧',
 ' 加保利 加保扶 第滅寧 芬殺螨 芬殺',
 '',
 ' 護粒松 加普胺 保米賜圃 矽酸爐渣 加普',
 ' 益達胺 鏈四環黴素 克枯爛 可尼丁 賽速安 派滅淨 四環黴素',
 ' 加保扶 陶斯松 賽洛寧 賜諾特 丁基加保扶 達特南 亞滅培 亞滅寧 滅賜克 丁基加保扶',
 ' 亞磷酸 微生物製劑 亞磷酸+氫氧化鉀',
 ' 第滅寧 

In [20]:
def Cosine_similarity(vec_a,vec_b):
  # Dot and norm
  dot = sum(a*b for a, b in zip(vec_a, vec_b))
  norm_a = sum(a*a for a in vec_a) ** 0.5
  norm_b = sum(b*b for b in vec_b) ** 0.5

  # Cosine similarity
  cos_sim = dot / (norm_a*norm_b)
  return cos_sim

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
#分次做：Txtcroplist
document_term_matrix = vectorizer.fit_transform(Txtcroplist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
Df999=(pdkey==0).all(axis=1)
#df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in Df999[Df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of Df
txtnumlist=[i for i in Df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

Df_sim_value1=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
Df_sim_value1.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,101,1002,1013,1059,1071,1026,1057,1058,1030,1035,1021,1055,1042,1003,1069,1020,106,100,1017,1047,1044,1018,1043,1031,1065,1046,1045,1077,1136,1183,1099,1139,1145,1160,11,1222,1102,1115,115,1090,...,837,719,883,657,623,900,832,934,955,932,971,944,952,931,958,957,920,945,995,964,954,99,962,983,914,942,947,94,982,978,981,959,922,989,974,976,966,948,98,953
101,1.000000,0.094537,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.362316,0.000000,0.362316,0.0,0.000000,0.362316,1.000000,0.0,0.195354,0.000000,0.362316,0.362316,0.0,0.362316,0.000000,0.000000,0.362316,0.362316,0.000000,0.0,0.000000,0.0,0.0,0.0,0.362316,0.0,0.362316,0.000000,0.362316,...,0.195354,0.0,0.107766,0.0,0.0,1.000000,0.195354,1.000000,0.000000,0.539382,1.000000,0.485512,0.000000,0.818770,0.092125,0.362316,0.870297,0.818770,0.870297,0.166321,0.552538,1.000000,0.0,0.362316,0.870297,0.870297,1.000000,1.000000,0.437550,0.362316,0.362316,0.166321,0.870297,0.0,0.870297,0.362316,0.0,0.678235,0.818770,0.0
1002,0.094537,1.000000,0.234645,0.0,0.0,0.000000,0.492829,0.000000,0.0,0.62249,0.281994,0.260923,0.233872,0.260923,0.0,0.668301,0.260923,0.094537,0.0,0.140685,0.492829,0.260923,0.260923,0.0,0.260923,0.492829,0.101936,0.260923,0.260923,0.379787,0.0,0.137634,0.0,0.0,0.0,0.260923,0.0,0.260923,0.316401,0.260923,...,0.140685,0.0,0.421349,0.0,0.0,0.094537,0.140685,0.094537,0.492829,0.000000,0.094537,0.000000,0.000000,0.077404,0.360196,0.260923,0.082275,0.077404,0.082275,0.119777,0.000000,0.094537,0.0,0.260923,0.082275,0.082275,0.094537,0.094537,0.041365,0.260923,0.260923,0.119777,0.082275,0.0,0.082275,0.260923,0.0,0.064118,0.077404,0.0
1013,0.000000,0.234645,1.000000,0.0,0.0,0.753015,0.000000,0.288359,0.0,0.00000,0.832094,0.000000,0.181987,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.188093,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.267481,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.306793,0.554635,0.000000,0.553527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1059,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.402912,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1071,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [22]:
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
#分次做：Txtpestlist
document_term_matrix = vectorizer.fit_transform(Txtpestlist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
Df999=(pdkey==0).all(axis=1)
#df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in Df999[Df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of Df
txtnumlist=[i for i in Df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

Df_sim_value2=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
Df_sim_value2.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,101,1002,1013,1059,1071,1026,1057,1058,1030,1035,1021,1055,1042,1003,1069,1020,106,100,1017,1047,1044,1018,1043,1031,1065,1046,1045,1077,1136,1183,1099,1139,1145,1160,11,1222,1102,1115,115,1090,...,837,719,883,657,623,900,832,934,955,932,971,944,952,931,958,957,920,945,995,964,954,99,962,983,914,942,947,94,982,978,981,959,922,989,974,976,966,948,98,953
101,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.847357,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.353389,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000
1002,0.0,1.000000,0.306754,0.0,0.0,0.306754,0.0,0.0,0.306754,0.218012,0.306754,0.128393,0.38027,0.138571,0.198743,0.218012,0.128393,0.128393,0.0,0.070994,0.0,0.198743,0.068180,0.0,0.128393,0.0,0.372239,0.128393,0.113545,0.0,0.128393,0.067035,0.0,0.243717,0.0,0.113545,0.128393,0.0,0.0,0.059119,...,0.0,0.136457,0.0,0.0,0.0,0.0,0.05545,0.0,0.0,0.0,0.717449,0.306754,0.168745,0.128393,0.057693,0.128393,0.0,0.128393,0.0,0.113545,0.000000,0.128393,0.0,0.0,0.0,0.0,0.0,0.0,0.479086,0.0,0.0,0.059119,0.0,0.0,0.0,0.0,0.306754,0.026209,0.0,0.32733
1013,0.0,0.306754,1.000000,0.0,0.0,1.000000,0.0,0.0,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.444840,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.00000
1059,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.625977,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000
1071,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000


In [23]:
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
#分次做：Txtchemlist
document_term_matrix = vectorizer.fit_transform(Txtchemlist) 
tfidfs = document_term_matrix.toarray()
tokens = vectorizer.get_feature_names()
pdkey=pd.DataFrame(tfidfs,columns=tokens)
#pdkey.head()

#確認有無全0的向量值
Df999=(pdkey==0).all(axis=1)
#Df999[df999==True].index

#設定全0的向量 其中第一格為0.0000000001
for i in Df999[Df999==True].index:
  if i==0:
    pdkey[pdkey.columns[0]][i]=pdkey[pdkey.columns[0]][i]+0.0000000001
  elif (i/10)<10:
    x=i+1
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i
  else:
    x=round(i/10)+int(i/10)
    pdkey[pdkey.columns[x]][i]=pdkey[pdkey.columns[x]][i]+0.0000000001*i

#為了要兩兩比較向量。 將pdkey轉成list格式(vector)，再每個ele叫出來兩兩比對
vec_list=[]
for i in range(pdkey.shape[0]):
  vec_list.append(list(pdkey.iloc[i,:]))

#print(vec_list[:2])
#print(len(vec_list))

ANS=[]
for i in range(len(vec_list)):
  ans=[]
  for j in range(len(vec_list)):
      value=Cosine_similarity(vec_list[i],vec_list[j])
      ans.append(value)

  ANS.append(ans)

#preprocessing of header of Df
txtnumlist=[i for i in Df['file_name']]
txtnumlist=[i.replace('.txt','') for i in txtnumlist]

Df_sim_value3=pd.DataFrame(np.array(ANS),columns=txtnumlist, index=txtnumlist)
Df_sim_value3.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,101,1002,1013,1059,1071,1026,1057,1058,1030,1035,1021,1055,1042,1003,1069,1020,106,100,1017,1047,1044,1018,1043,1031,1065,1046,1045,1077,1136,1183,1099,1139,1145,1160,11,1222,1102,1115,115,1090,...,837,719,883,657,623,900,832,934,955,932,971,944,952,931,958,957,920,945,995,964,954,99,962,983,914,942,947,94,982,978,981,959,922,989,974,976,966,948,98,953
101,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.162986,0.0,0.0,0.443588,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.480593,0.000000,0.0,0.0,0.0,0.22651,0.301564,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.368307,0.0,0.480593,0.0,0.0,0.0,0.0,0.0,0.22651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210664,0.0,0.000000,0.0,0.0,0.0
1002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.425406,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1013,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.798755,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.667705,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.368265,0.0,0.0,0.0
1059,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.372282,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1071,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.352377,0.0,0.0,0.0,0.0,0.0,0.185006,0.0,0.000000,0.0,0.0,0.258763,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.494947,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.080633,0.0,0.0,0.0


In [24]:
#把df轉成table列表
#sim_value_Crop, sim_value_Pest, sim_value_Chem
Df1=Df_sim_value1.where(np.triu(np.ones(Df_sim_value1.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Crop')
Df2=Df_sim_value1.where(np.tril(np.ones(Df_sim_value1.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Crop')
Df3=Df_sim_value2.where(np.triu(np.ones(Df_sim_value2.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Pest')
Df4=Df_sim_value2.where(np.tril(np.ones(Df_sim_value2.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Pest')
Df5=Df_sim_value3.where(np.triu(np.ones(Df_sim_value3.shape, dtype=bool),1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Chem')
Df6=Df_sim_value3.where(np.tril(np.ones(Df_sim_value3.shape, dtype=bool),-1).astype(np.bool)).stack().rename_axis(('Test', 'Reference')).reset_index(name='sim_value_Chem')
Tablesim1 = pd.concat([Df1,Df2])
Tablesim2 = pd.concat([Df3,Df4])
Tablesim3 = pd.concat([Df5,Df6])
#tablesim
Tablesim1.reset_index(drop=True, inplace=True)
Tablesim2.reset_index(drop=True, inplace=True)
Tablesim3.reset_index(drop=True, inplace=True)
#暫存各個分類檔案
Tablesim1.to_pickle('Tablesim_Crop')
Tablesim2.to_pickle('Tablesim_Pest')
Tablesim3.to_pickle('Tablesim_Chem')

In [25]:
#合併
TabelCrop=pd.read_pickle('Tablesim_Crop')
TabelPest=pd.read_pickle('Tablesim_Pest')
TabelChem=pd.read_pickle('Tablesim_Chem')
Tablesim=pd.concat([TabelCrop,TabelPest['sim_value_Pest'],TabelChem['sim_value_Chem'] ],axis=1)

#加入關鍵字詞數
Tablesim['Test']=Tablesim['Test'].astype(int)
Tablesim['Reference']=Tablesim['Reference'].astype(int)

Tablesim['Test_key_cou']=np.nan
for i in range(Tablesim.shape[0]):
  Tablesim['Test_key_cou'][i]=Df[Df['name_num']==Tablesim['Test'][i]]['keycount']

Tablesim['Ref_key_cou']=np.nan
for i in range(Tablesim.shape[0]):
  Tablesim['Ref_key_cou'][i]=Df[Df['name_num']==Tablesim['Reference'][i]]['keycount']

Tablesim.head()

串流輸出內容已截斷至最後 5000 行。
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A

,Test,Reference,sim_value_Crop,sim_value_Pest,sim_value_Chem,Test_key_cou,Ref_key_cou
0,101,1002,0.094537,0.0,0.0,8.0,13.0
1,101,1013,0.000000,0.0,0.0,8.0,14.0
2,101,1059,0.000000,0.0,0.0,8.0,5.0
3,101,1071,0.000000,0.0,0.0,8.0,12.0
4,101,1026,0.000000,0.0,0.0,8.0,6.0


In [26]:
Tablesim.reset_index(drop=True,inplace=True)
Tablesim['comb']=Tablesim['Test'].astype(str)+'_'+Tablesim['Reference'].astype(str)
Tablesim

,Test,Reference,sim_value_Crop,sim_value_Pest,sim_value_Chem,Test_key_cou,Ref_key_cou,comb
0,101,1002,0.094537,0.0,0.0,8.0,13.0,101_1002
1,101,1013,0.000000,0.0,0.0,8.0,14.0,101_1013
2,101,1059,0.000000,0.0,0.0,8.0,5.0,101_1059
3,101,1071,0.000000,0.0,0.0,8.0,12.0,101_1071
4,101,1026,0.000000,0.0,0.0,8.0,6.0,101_1026
...,...,...,...,...,...,...,...,...
175975,953,974,0.000000,0.0,0.0,5.0,13.0,953_974
175976,953,976,0.000000,0.0,1.0,5.0,4.0,953_976
175977,953,966,0.000000,0.0,0.0,5.0,15.0,953_966
175978,953,948,0.000000,0.0,1.0,5.0,16.0,953_948


In [132]:
#先篩選條件 
#1.先篩掉3個都0的 
Df_del=Tablesim[(Tablesim['sim_value_Crop']==0)&(Tablesim['sim_value_Pest']==0)&(Tablesim['sim_value_Chem']==0)]
print(f'刪除的{Df_del.shape[0]}')
Df_mining1=Tablesim.drop(Df_del.index)
Df_mining1.reset_index(inplace=True,drop=True)

#2.刪掉Crop跟Pest是0的
Df_de2=Df_mining1[(Df_mining1['sim_value_Crop']==0)&(Df_mining1['sim_value_Pest']==0)]
print(f'刪除的{Df_de2.shape[0]}')
Df_mining2=Df_mining1.drop(Df_de2.index)
Df_mining2.reset_index(inplace=True,drop=True)

#3.sim_crop + sim_pest + sim_chem > 2 and Test_key_cou <= Ref_key_cou / sim_crop + sim_pest + 
Df_mining3=Df_mining2[((Df_mining2['sim_value_Crop']+Df_mining2['sim_value_Pest']+Df_mining2['sim_value_Chem'])>2.3)
            &(Df_mining2['Test_key_cou']<=Df_mining2['Ref_key_cou'])]
#Df_mining3=Df_mining2[(Df_mining2['sim_value_Crop']>0.5)&(Df_mining2['sim_value_Pest']>0.5)&(Df_mining2['sim_value_Chem']>0.5)]
print(f'剩下的{Df_mining3.shape[0]}')




刪除的112846
刪除的10112
剩下的845


In [133]:
Df_mining3.reset_index(inplace=True,drop=True)
Df_mining3


,Test,Reference,sim_value_Crop,sim_value_Pest,sim_value_Chem,Test_key_cou,Ref_key_cou,comb
0,1026,1021,0.414012,1.000000,1.000000,6.0,13.0,1026_1021
1,1026,481,0.626935,1.000000,1.000000,6.0,9.0,1026_481
2,1057,115,0.642010,1.000000,0.765868,10.0,15.0,1057_115
3,1057,173,0.770626,1.000000,0.784554,10.0,12.0,1057_173
4,1057,378,1.000000,1.000000,0.452052,10.0,12.0,1057_378
...,...,...,...,...,...,...,...,...
840,914,900,0.870297,1.000000,1.000000,12.0,17.0,914_900
841,981,1359,0.539181,1.000000,0.796619,17.0,18.0,981_1359
842,981,599,1.000000,1.000000,0.674233,17.0,20.0,981_599
843,981,555,1.000000,0.723449,0.622200,17.0,22.0,981_555


In [134]:
#生成testPrivatedata_1214
Df_mining3.to_pickle('testPrivatedata_1214')

##三、prediction

載入模型

In [135]:
import joblib
#model1='/content/drive/MyDrive/ColabNotebooks/AIProject_Similarity/Tree_1213.model'
Tree = joblib.load("Tree_1214.model")

In [136]:
# Classifying new data
test_data = pd.read_pickle("testPrivatedata_1214")  # read test data
test_input = np.array(test_data[['sim_value_Crop','sim_value_Pest','sim_value_Chem','Test_key_cou','Ref_key_cou']])

In [137]:
yhat=Tree.predict(test_input)
predict_class = le.inverse_transform(yhat)
df_predict= pd.DataFrame(predict_class , columns=['Label'])
df_predict['Test']=test_data['Test']
df_predict['Reference']=test_data['Reference']
df_predict.head()

,Label,Test,Reference
0,Yes,1026,1021
1,Yes,1026,481
2,Yes,1057,115
3,Yes,1057,173
4,Yes,1057,378


In [138]:
#挑出yes的
Df_predict=df_predict[df_predict['Label']=='Yes']
Df_predict['comb']=Df_predict['Test'].astype(str)+'_'+Df_predict['Reference'].astype(str)
Df_predict

,Label,Test,Reference,comb
0,Yes,1026,1021,1026_1021
1,Yes,1026,481,1026_481
2,Yes,1057,115,1057_115
3,Yes,1057,173,1057_173
4,Yes,1057,378,1057_378
...,...,...,...,...
840,Yes,914,900,914_900
841,Yes,981,1359,981_1359
842,Yes,981,599,981_599
843,Yes,981,555,981_555


In [139]:
Df_predict.to_csv("1213_845.csv")

##四、selection

####5823預測
